## Побудова то оцінка базових моделей

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import sklearn.preprocessing as prep
import warnings
import logging
import sklearn.covariance as cov
from scipy.stats import chi2
from itertools import product
import seaborn as sns
import random
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.naive_bayes import GaussianNB  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from numpy import hstack

# відключення логерів та попереджень
warnings.filterwarnings('ignore')
logging.getLogger('matplotlib.font_manager').disabled = True
log = logging.getLogger()
log.setLevel(logging.CRITICAL)

pd.set_option('display.max_columns', None)

In [2]:
cleaned_data = pd.read_pickle('cleaned_data_2.pkl')
cleaned_data.drop(columns=['FLAG_MOBIL', 'FLAG_DOCUMENT_20'], inplace=True)  # константи

In [3]:
metric_df = cleaned_data.loc[:,(np.array(cleaned_data.dtypes == "float64")) | (np.array(cleaned_data.dtypes == "int64"))].copy()
non_metric_df = cleaned_data.loc[:,(np.array(cleaned_data.dtypes !="float64")) & (np.array(cleaned_data.dtypes != "int64"))].copy()

### Кодування категорійних змінних

Деякі моделі класифікації не можуть працювати з категорійними змінними, тому необхідно кодування змінних.    
* Змінні з лише двома значеннями закодуємо _LabelEncoder_.       
* Змінні з більшою кількістю значень закодуємо _OneHotEncoder_    

In [4]:
col_less = [item for item in non_metric_df.columns if cleaned_data[item].nunique() < 3]
df_less = cleaned_data[col_less]
col_ns = [item for item in non_metric_df.columns if cleaned_data[item].nunique() > 2]
df_ns = cleaned_data[col_ns]

In [5]:
le = LabelEncoder()
for col in col_less:
    df_less[col] = le.fit_transform(df_less[col])

In [6]:
df_ns = pd.get_dummies(df_ns)

In [7]:
df = metric_df.merge(df_less, left_index=True, right_index=True).merge(df_ns, left_index=True, right_index=True)

### Побудова базових моделей

__Стандартизація метричних змінних__

In [8]:
y = df[['TARGET']].values
X = df.drop(columns=['TARGET'])
se = StandardScaler()
X = se.fit_transform(X)
X = normalize(X)

__Виділення тренувального та тестового датасетів__

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [10]:
models = {}

__Логістична регресія__

In [11]:
clf = LogisticRegression(random_state=0, C = 0.01, n_jobs=6)
clf.fit(X_train, y_train)
models.update({'LR': clf})

__Градієнтний бустінг випадкового лісу (XGBoost)__

In [12]:
clf = xgb.XGBClassifier(n_jobs=6, verbosity=0)
clf.fit(X_train, y_train, verbose=False, early_stopping_rounds=100, eval_metric="auc",
        eval_set=[(X_test, y_test)])
models.update({'XGB': clf})

__Градієнтний бустінг випадкового лісу (LGBM)__

In [13]:
clf = LGBMClassifier(n_jobs = 6)
clf.fit(X_train, y_train)
models.update({'LGBM': clf})

__Лінійний дискримінантний аналіз__

In [14]:
clf = LinearDiscriminantAnalysis(solver='lsqr')
clf.fit(X_train, y_train)
models.update({'LDA': clf})

__Байєсівський класифікатор__

In [15]:
clf = GaussianNB()
clf.fit(X_train, y_train)
models.update({'NB': clf})

__Класифікатор дерева рішень__

In [16]:
clf = DecisionTreeClassifier(criterion='entropy', )
clf.fit(X_train, y_train)
models.update({'DT': clf})

__Порівняння моделей__

In [17]:
for key in models.keys():
    clf = models[key]
    y_proba = clf.predict_proba(X_test)[:, 1]
    y_pred = clf.predict(X_test)
    print('{}: AUC {}, Accuracy score {}'.format(key, roc_auc_score(y_test, y_proba), accuracy_score(y_test, y_pred)))

LR: AUC 0.7368163673329833, Accuracy score 0.9179859394449412
XGB: AUC 0.7433183556531034, Accuracy score 0.9177755570749838
LGBM: AUC 0.7456149856958607, Accuracy score 0.9177404933466575
LDA: AUC 0.7368610370394694, Accuracy score 0.9172671330142534
NB: AUC 0.5427182853876715, Accuracy score 0.14092112414313013
DT: AUC 0.5345904246118365, Accuracy score 0.8530479145847578


In [18]:
def fit_ensemble(models, X_train, X_val, y_train, y_val):
    meta_X = list()
    for model in models:
        model.fit(X_train, y_train)
        yhat = model.predict(X_val)
        yhat = yhat.reshape(len(yhat), 1)
        meta_X.append(yhat)
    meta_X = hstack(meta_X)
    blender = LogisticRegression()
    blender.fit(meta_X, y_val)
    return blender

In [19]:
def predict_ensemble(models, blender, X_test):
    meta_X = list()
    for model in models:
        yhat = model.predict(X_test)
        yhat = yhat.reshape(len(yhat), 1)
        meta_X.append(yhat)
    meta_X = hstack(meta_X)
    return blender.predict(meta_X)

In [20]:
# split dataset into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
# split training set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.33, random_state=1)
# summarize data split
print('Train: %s, Val: %s, Test: %s' % (X_train.shape, X_val.shape, X_test.shape))

Train: (95539, 217), Val: (47058, 217), Test: (142598, 217)


In [22]:
models = {models[key] for key in list(models.keys())[:4]}

In [23]:
# train the blending ensemble
blender = fit_ensemble(models, X_train, X_val, y_train, y_val)
# make predictions on test set
yhat = predict_ensemble(models, blender, X_test)
score = accuracy_score(y_test, yhat)
print('Blending Accuracy: {:.3f}'.format(score))

Blending Accuracy: 0.919
